# Mining testimonial fragments of the Holocaust

**Experience domain:** numbness

### Load the necessary libraries

In [363]:
import sys; sys.path.insert(0, '..')
import itertools

In [364]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text

In [365]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [87]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [337]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [174]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [177]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [182]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [302]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [300]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [298]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Set up the query

In [55]:
query = '[lemma="numb"]'

In [10]:
result = topic_concordancer.main(query,window=50,topicn=25)

Training lda model began
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5Blemma%3D%22numb%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=50


### Print the key topics

In [11]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

0
0.215*"shock" + 0.190*"mind" + 0.152*"moment" + 0.127*"die" + 0.127*"Germans" + 0.114*"understand" + 0.038*"come_back" + 0.025*"child" + 0.013*"shoot" + 0.000*"march"


1
0.703*"time" + 0.108*"kill" + 0.095*"try_to" + 0.041*"thing" + 0.027*"Jews" + 0.014*"end_of" + 0.014*"leave" + 0.000*"march" + 0.000*"of_course" + 0.000*"my_mother"


2
0.955*"feel" + 0.027*"dead" + 0.018*"march" + 0.000*"never" + 0.000*"stand" + 0.000*"Jews" + 0.000*"leave" + 0.000*"my_mother" + 0.000*"person" + 0.000*"foot"


3
0.357*"become_numb" + 0.167*"lose" + 0.107*"realize" + 0.107*"hope" + 0.095*"move" + 0.095*"try_to" + 0.036*"live" + 0.024*"of_course" + 0.012*"give" + 0.000*"my_mother"


4
0.513*"feeling" + 0.211*"place" + 0.197*"completely" + 0.039*"do_not" + 0.039*"come_back" + 0.000*"of_course" + 0.000*"leave" + 0.000*"march" + 0.000*"my_mother" + 0.000*"never"


5
0.392*"thing" + 0.373*"happen" + 0.176*"sit" + 0.049*"die" + 0.010*"time" + 0.000*"walk" + 0.000*"foot" + 0.000*"my_mother" + 0.000*"never"

### Analyze documents

In [12]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

the shock , the variation of shocks and remember I was arrested in 1942 , so I was n't even 16 years old yet , i was 15 year old . My sister was not even 12 , not even 13 she was , well 1928 . We were so numb , we were almost , the shock hypnotized us , I believe and for that matter , I 'm slowly getting out of hypnosis right now , in 1996 , even now , today ... [ end of side 1 of tape 1 ] Q : So let 's go 


. And I , when I heard about one classmate who had died already , this was like three months later , it was still a normal reaction , because I had just come from a normal , more or less normal environment , so that the emotions were not numb yet . But , as things were happening , you were in shock , and you , you did not have too much time to think . You just had to act upon all the orders that were constantly being given to you by the Germans . And , of 


a barrack with strangers . Change of tape INT : Irene Zisblatt , take five . Irene , you described your experiment -- the ex

## Testimonial fragments

### 1.  Kill

In [64]:
lemmas = ['kill','numb']

In [65]:
query = create_contextual_query(lemmas,context_length=50)

In [66]:
print (query)

([lemma="kill"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="kill"])


In [49]:
domain_term = 'kill'

In [ ]:
fragments = []

In [67]:
fragment_1 = {}
fragment_1['original_sentence'] = "[PAUSES FOR 3 SECONDS] Then they said, you're out of luck because we're going to kill you all. [PAUSES FOR 4 SECONDS] We [PAUSES FOR 4 SECONDS] were numb."
fragment_1['fragment_label']="Then they said, you're out of luck because we're going to kill you all. We were numb."

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = ''
fragment_2['fragment_label']='We were so numb, we were almost, the shock hypnotized us (..)'

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = ''
fragment_2['fragment_label']='To see another 10,000 Jews come and be killed, another 20,000 be killed? (..) Then after awhile you become so numb (..)

In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = ''
fragment_3['fragment_label']="You are numb. You're shocked"

In [45]:
fragment_4 = {}
fragment_4['original_sentence'] = 'And we were so numb, [? Cathy ?] every 10 people get killed.'
fragment_4['fragment_label']="And we were so numb, (..) every 10 people get killed."

In [71]:
fragment_5 = {}
fragment_5['original_sentence'] = "And I-- I-- I heard screams and-- from-- from-- from-- from people being wounded, from people being killed, from cries. And finally, everything subsided. But I was alive. I didn't-- I didn't even think. I was numb. And finally, everything subsided. But I was alive. I didn't-- I didn't even think. I was numb."



In [ ]:
fragment_5['fragment_label']="I heard screams (..) from people being wounded, from people being killed, from cries. (..) I was numb."

### 2.  Shock

In [ ]:
query = '[lemma="shock"]([]{0,25}[lemma="numb"])|[lemma="numb"]([]{0,25}[lemma="shock"])'

In [42]:
domain_term = 'shock'

In [45]:
fragment_1 = {}
fragment_1['original_sentence'] = 'c'
fragment_1['fragment_label']='I was in complete shock. I was numb, nothing.'

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = ''
fragment_2['label']='We were so numb, we were almost, the shock hypnotized us (..)'

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = ''
fragment_2['fragment_label']=''

In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = ''
fragment_3['fragment_label']="You are numb. You're shocked"

In [45]:
fragment_4 = {}
fragment_4['original_sentence'] = ''
fragment_4['fragment_label']="I was numb when I was there. I'll be very honest with you. I was not-- I was somewhat in a state of shock."

In [45]:
fragment_5 = {}
fragment_5['original_sentence'] = ''
fragment_5['fragment_label']='I was a completely-- I guess I was-- I was 15. I was shocked. I was, uh, numb.'

### 3.  Dead inside

In [303]:
main_node = "numbness"
add_main_node('numbness')

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


In [84]:
lemmas = ['lose','numb']

In [85]:
query = create_contextual_query(lemmas,context_length=50)

In [86]:
print (query)

([lemma="alive"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="alive"])


In [355]:
fragments = {}

In [356]:
fragments['main_node'] = main_node

In [357]:
fragments['mid_node'] = 'dead inside'
fragments['fragments']= []

{'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [358]:
fragment_1 = {}
fragment_1['original_sentence'] = 'By the time the Nazis took us into the concentration camp I was so numb inside, I was so dead, I was not able to feel anything.'
fragment_1['label']='(..) I was so numb inside, I was so dead (..)'
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22By%22%5D+%5B%22the%22%5D+%5B%22time%22%5D+%5B%22the%22%5D+%5B%22Nazis%22%5D+%5B%22took%22%5D+%5B%22us%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22concentration%22%5D+%5B%22camp%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22inside%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22dead%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%22able%22%5D+%5B%22to%22%5D+%5B%22feel%22%5D+%5B%22anything%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'By the time the Nazis took us into the concentration camp I was so numb inside , I was so dead , I was not able to feel anything . ', 'right': '', 'complete_match': 'By the time the Nazis took us into the concentration camp I was so numb inside , I was so dead , I was not able to feel anything . ', 'testimony_id': 'irn504792', 'shelfmark': ['USHMM RG-50.030*0296'], 'token_s

In [359]:
fragment_2 = {}
fragment_2['original_sentence'] = 'You were so numb that everything that you saw around you, that you, your feelings were dead.'
fragment_2['label']='You were so numb that everything that you saw around you, that you, your feelings were dead.'
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22everything%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22saw%22%5D+%5B%22around%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22your%22%5D+%5B%22feelings%22%5D+%5B%22were%22%5D+%5B%22dead%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You were so numb that everything that you saw around you , that you , your feelings were dead . ', 'right': '', 'complete_match': 'You were so numb that everything that you saw around you , that you , your feelings were dead . ', 'testimony_id': 'irn505576', 'shelfmark': ['USHMM RG-50.042*0023'], 'token_start': 8257, 'token_end': 8277}


In [360]:
fragment_3 = {}
fragment_3['original_sentence'] = 'Uh, we had, we were moving sometimes uh, fifty or sixty miles a day and working day and night and uh, we were just uh, uh, we were just dead on our feet, and uh, one of the problems that we had in relation to this type of recollection is that uh, we were numb.'
fragment_3['label']='we were just dead on our feet, and uh, one of the problems that we had in relation to this type of recollection is that uh, we were numb.'
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22moving%22%5D+%5B%22sometimes%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22fifty%22%5D+%5B%22or%22%5D+%5B%22sixty%22%5D+%5B%22miles%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%22and%22%5D+%5B%22working%22%5D+%5B%22day%22%5D+%5B%22and%22%5D+%5B%22night%22%5D+%5B%22and%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22dead%22%5D+%5B%22on%22%5D+%5B%22our%22%5D+%5B%22feet%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22problems%22%5D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%22in%22%5D+%5B%22relation%22%5D+%5B%22to%22%5D+%5B%22this%22%5D+%5B%22type%22%5D+%5B%22of%22%5D+%5B%22recollection%22%5D+%5B%22is%22%5D+%5B%22that%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5

In [361]:
fragment_4 = {}
fragment_4['original_sentence'] = 'Your heart, everything, your whole body start getting already like numb and dead.'
fragment_4['label']="Your heart, everything, your whole body start getting already like numb and dead."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Your%22%5D+%5B%22heart%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%5D+%5B%22your%22%5D+%5B%22whole%22%5D+%5B%22body%22%5D+%5B%22start%22%5D+%5B%22getting%22%5D+%5B%22already%22%5D+%5B%22like%22%5D+%5B%22numb%22%5D+%5B%22and%22%5D+%5B%22dead%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Your heart , everything , your whole body start getting already like numb and dead . ', 'right': '', 'complete_match': 'Your heart , everything , your whole body start getting already like numb and dead . ', 'testimony_id': 'irn510703', 'shelfmark': ['USHMM RG-50.156*0049'], 'token_start': 1038, 'token_end': 1054}


In [362]:
add_testimonial_fragments(fragments)

### 4.  Lose

In [95]:
lemmas =['lose','numb']

In [96]:
query = create_contextual_query(lemmas)

In [97]:
print (query)

([lemma="lose"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="lose"])


In [42]:
domain_term = 'lose'

In [98]:
fragment_1 = {}
fragment_1['original_sentence'] = "I saw parents who lost their children. I saw husbands who lost their wives and vise versa, and they were numbed, actually, to-- they didn't complain."
fragment_1['fragment_label']="I saw parents who lost their children. I saw husbands who lost their wives and vise versa, and they were numbed (..)"

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = "2:3:18 The most tragic kind of family is the numb family who are survivors who usually lost both their parents and their children."
fragment_2['fragment_label']="(..) the numb family who are survivors who usually lost both their parents and their children."

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = "MR: I was so numb that I don't even know. I was all numb. I lost everybody."
fragment_2['fragment_label']="I was all numb. I lost everybody."

In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = "I still was despising...but I had become somehow numb. I knew I couldn't change things...all these people I have lost, all these victims I had seen, all these corpses I had seen lying around, and by the way I lost also then within a short time, two of my close friends be...because they they could...one was a man who was highly idealistic and he couldn't take the misery and the the this this this this height of inhumanity, so he he uh said enough is enough."
fragment_3['fragment_label']="I had become somehow numb. I knew I couldn't change things...all these people I have lost"

### 5.  No feeling

In [99]:
lemmas =['feeling','numb']

In [100]:
query = create_contextual_query(lemmas)

In [101]:
print (query)

([lemma="feeling"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="feeling"])


In [102]:
domain_term = 'no feeling'

In [98]:
fragment_1 = {}
fragment_1['original_sentence'] = "We were numb. The five years there were no feelings."
fragment_1['fragment_label']="We were numb. The five years there were no feelings."

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = "They had no feeling to even think how I would feel. And so I'm just like a machine, numb machine, going through the mechanical motions of something that I was forced into doing."
fragment_2['fragment_label']="They had no feeling to even think how I would feel. And so I'm just like a machine, numb machine (..)"

In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = "But you become numb, you know. You...you're not really sensitive to feelings any more."
fragment_3['fragment_label']="But you become numb, you know. You...you're not really sensitive to feelings any more."

In [103]:
fragment_4 = {}
fragment_4['original_sentence'] = "The were no feelings. You were completely numb."
fragment_4['fragment_label']= "The were no feelings. You were completely numb."

In [105]:
fragment_5 = {}
fragment_5['original_sentence'] = "It was all numb and just – couldn’t have any feeling, it felt like just you – you know how your leg – how something goes to sleep when you’re [indecipherable] that’s the way it felt."
fragment_5['fragment_label']= "It was all numb and just – couldn’t have any feeling"


### 6.  No cry

In [111]:
lemmas =['cry','numb']

In [112]:
query = create_contextual_query(lemmas)

In [113]:
print (query)

([lemma="cry"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="cry"])


In [114]:
domain_term = 'cry'

In [98]:
fragment_1 = {}
fragment_1['original_sentence'] = "You never cried. You so...it was so numb...it's...it's very hard to explain."
fragment_1['fragment_label']="You never cried. You so...it was so numb...it's...it's very hard to explain."

In [45]:
fragment_2 = {}
fragment_2['original_sentence'] = "That was--we were numb. We couldn't--it wasn't a question--we weren't crying but we were--we were like num--just pieces of would."
fragment_2['fragment_label']="(..)we were numb. We couldn't--it wasn't a question--we weren't crying (..)"

In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = "But I-- at that time, I was like-- I felt like I, I was-- I couldn't even cry anymore. Just couldn't. I mean, there just-- I was numb."
fragment_3['fragment_label']="I couldn't even cry anymore. Just couldn't. I mean, there just-- I was numb."

In [115]:
fragment_4 = {}
fragment_4['original_sentence'] = "And after about four hours-- and I probably was numb; I couldn't cry or say [?"
fragment_4['fragment_label']="I probably was numb; I couldn't cry"

In [ ]:
fragment_5 = {}
fragment_5['original_sentence'] = "I was numb. I didn’t express any emotions or anything like that, I didn’t cry."
fragment_5['fragment_label']="I was numb. I didn’t express any emotions or anything like that, I didn’t cry."